# Question 1

Probabilité d'aimer (score >= 4) toy story sachant que je suis une femme ingénieure de plus de 52 ans (age >= 30: non-jeune). On suppose l'indépendance des facteurs

P(score >= 4 | sexe = 'F', age >= 30,  job = 'ing') = P(score >= 4) * P(sexe = 'F' | score >= 4) * P(age >= 30 | score >= 4) * P(job = 'ing' | score >= 4)

T: (total nombre de reviews pour toy story)

W: (nombre de personnes ingénieurs qui ont donné un score >= 4 à toy story)

X: (nombres de scores >= 4 pour toy story dans notre tableau)

Y: (nombre de personnes de sexe 'F' qui ont donné un score >= 4 à toy story)

Z: (nombre de personnes d'égal ou de plus de 30 ans qui ont donné un score >= 4 à toy story)

P(score >= 4): X / T

P(sexe = 'F' | score >= 4): Y / X

P(age >= 30 | score >= 4): Z / X

P(job = 'ing' | score >= 4): W / X

Réponse: P(score >= 4 | sexe = 'F', age >= 30) = (X / T) * (Y / X) * (Z / X) * (W / X) = Y * Z * W / (T * X^2)



In [1]:
import pandas as pd
import numpy as np
import scipy.io

# Load RData file (requires conversion)
votes = pd.read_csv('./data/votes.csv')
users = pd.read_csv('./data/u.csv')
items = pd.read_csv('./data/items.csv')

In [2]:
toy_story_id = items[items['movie.title'].str.contains('Toy Story', case=False, na=False)]['movie.id'].values[0]
toy_story_votes = votes[votes['item.id'] == toy_story_id]

n_ts_votes = len(toy_story_votes)
toy_story_votes_liked = toy_story_votes[toy_story_votes['rating'] >= 4.0]
n_ts_votes_liked = len(toy_story_votes_liked)

prob_liked = n_ts_votes_liked / n_ts_votes

print(f"nombre de votes pour toy story: {n_ts_votes}")
print(f"nombre de votes pour toy story en haut ou égal à 4: {n_ts_votes_liked}")
print(f"probabilité d'avoir un vote de toy story en haut ou égal à 4: {n_ts_votes_liked / n_ts_votes}")


nombre de votes pour toy story: 452
nombre de votes pour toy story en haut ou égal à 4: 321
probabilité d'avoir un vote de toy story en haut ou égal à 4: 0.7101769911504425


In [3]:
merged_user_votes = pd.merge(toy_story_votes_liked, users, left_on='user.id', right_on='id', how='left')
female_votes_liked = merged_user_votes[merged_user_votes['gender'] == 'F']
eng_votes_liked = merged_user_votes[merged_user_votes['job'] == 'engineer']
nonjeune_votes_liked = merged_user_votes[merged_user_votes['age'] >= 30]

n_female_ts_votes_liked = len(female_votes_liked)
n_eng_ts_votes_liked = len(eng_votes_liked)
n_nonjeune_ts_votes_liked = len(nonjeune_votes_liked)

prob_female_liked = n_female_ts_votes_liked / n_ts_votes_liked
prob_eng_liked = n_eng_ts_votes_liked / n_ts_votes_liked
prob_nonjeune_liked = n_nonjeune_ts_votes_liked / n_ts_votes_liked


print(f"nombre de bon ratings femele pour toy story: {n_female_ts_votes_liked}")
print(f"nombre de bon ratings ingénieurs pour toy story: {n_eng_ts_votes_liked}")
print(f"nombre de bon ratings non-jeune pour toy story en haut ou égal à 4: {n_nonjeune_ts_votes_liked}")
print(f"probabilité d'avoir un vote femele de toy story en haut ou égal à 4: {n_female_ts_votes_liked / n_ts_votes_liked}")
print(f"probabilité d'avoir un vote ingénieur de toy story en haut ou égal à 4: {n_eng_ts_votes_liked / n_ts_votes_liked}")
print(f"probabilité d'avoir un vote non-jeune de toy story en haut ou égal à 4: {n_nonjeune_ts_votes_liked / n_ts_votes_liked}")

nombre de bon ratings femele pour toy story: 80
nombre de bon ratings ingénieurs pour toy story: 35
nombre de bon ratings non-jeune pour toy story en haut ou égal à 4: 164
probabilité d'avoir un vote femele de toy story en haut ou égal à 4: 0.24922118380062305
probabilité d'avoir un vote ingénieur de toy story en haut ou égal à 4: 0.10903426791277258
probabilité d'avoir un vote non-jeune de toy story en haut ou égal à 4: 0.5109034267912772


In [4]:
prob_total = prob_liked * prob_female_liked * prob_eng_liked * prob_nonjeune_liked
print(f"probabilité finale: {prob_total}")

probabilité finale: 0.009859465683949332


# Question 2

In [5]:
import numpy as np
import pandas as pd

def dcg_at_p(relevant_ratings, p):
    """
    Calculate the DCG at position p.
    """
    dcg = 0
    for i in range(p):
        if i < len(relevant_ratings):
            dcg += (relevant_ratings[i]) / np.log2(i + 2)  # 1-based index for log
    return dcg

def ndcg(actual_ratings, predicted_ratings, p=10):
    """
    Calculate the NDCG at position p.
    """
    # Sort predicted ratings and actual ratings
    predicted_ratings_sorted = predicted_ratings.argsort()[::-1]
    actual_ratings_sorted = actual_ratings.argsort()[::-1]
    
    # Get the top p items based on predicted ratings
    predicted_top_p = predicted_ratings_sorted[:p]
    actual_top_p = actual_ratings_sorted[:p]
    
    # Map the ratings to relevance levels (0 or 1 for relevance)
    predicted_relevance = actual_ratings[predicted_top_p]
    actual_relevance = actual_ratings[actual_top_p]
    
    # Compute DCG and IDCG
    dcg = dcg_at_p(predicted_relevance, p)
    idcg = dcg_at_p(sorted(actual_relevance, reverse=True), p)
    
    # Return the NDCG value
    return dcg / idcg if idcg > 0 else 0

votes = pd.read_csv('./data/votes.csv')
users = pd.read_csv('./data/u.csv')
items = pd.read_csv('./data/items.csv')

rating_matrix = votes.pivot(index='user.id', columns='item.id', values='rating')
#print(rating_matrix.head())

# 1. Load data (users as rows, items as columns)
ratings_preds_bayes = pd.read_csv("./predictions/pred-bayes.csv", index_col=0)  # Assumes first column is user_id
ratings_preds_bayes.index.name = 'movie.id'
ratings_preds_bayes.columns = ratings_preds_bayes.columns.str.replace('V', '').astype(int)
#print(ratings_preds_bayes.head())

ratings_preds_voisins = pd.read_csv("./predictions/pred-voisins-obs.csv")
ratings_preds_voisins.index.name = 'movie.id'
ratings_preds_voisins.columns = ratings_preds_voisins.columns.str.replace('V', '').astype(int)
ratings_preds_voisins.index = range(1, len(ratings_preds_voisins) + 1)
#print(ratings_preds_voisins.head())

# 2. Compute means
user_means = rating_matrix.mean(axis=1, skipna=True)  # Compute mean for each user (ignoring NaNs)
item_means = rating_matrix.mean(axis=0, skipna=True)  # Compute mean for each user (ignoring NaNs)

#print(user_means)

# 3. Predict missing values using user mean
predicted_matrix_user = rating_matrix.copy()
predicted_matrix_item = rating_matrix.copy()

for user in rating_matrix.index:
    predicted_matrix_user.loc[user] = user_means[user]  

for item in rating_matrix.columns:
    predicted_matrix_item[item] = item_means[item]

#print(predicted_matrix_user)
#print(predicted_matrix_item)

# 4. Compute NDCG for each user
ndcg_scores_user = []
for user in rating_matrix.index:
    true_ratings = rating_matrix.loc[user].dropna().values  # Get actual ratings
    predicted_ratings = predicted_matrix_user.loc[user, rating_matrix.loc[user].notna()].values  # Use predictions only for rated items

    if len(true_ratings) > 1:  # Ensure there's enough data for ranking
        ndcg_scores_user.append(ndcg(true_ratings, predicted_ratings))

ndcg_scores_item = []
for user in rating_matrix.index:
    true_ratings = rating_matrix.loc[user].dropna().values  # Get actual ratings
    predicted_ratings = predicted_matrix_item.loc[user, rating_matrix.loc[user].notna()].values  # Use predictions only for rated items

    if len(true_ratings) > 1:  # Ensure there's enough data for ranking
        ndcg_scores_item.append(ndcg(true_ratings, predicted_ratings))

#print(ratings_preds_bayes.loc[1])
#print(ratings_preds_bayes.index)
#print(ratings_preds_bayes.head())
ndcg_scores_bayes = []
for user in rating_matrix.index:
    true_ratings = rating_matrix.loc[user].dropna().values  # Get actual ratings
    predicted_ratings = ratings_preds_bayes.loc[user, rating_matrix.loc[user].notna()].values  # Use predictions only for rated items

    if len(true_ratings) > 1:  # Ensure there's enough data for ranking
        ndcg_scores_bayes.append(ndcg(true_ratings, predicted_ratings))

ndcg_scores_voisins = []
for user in rating_matrix.index:
    true_ratings = rating_matrix.loc[user].dropna().values  # Get actual ratings
    predicted_ratings = ratings_preds_voisins.loc[user, rating_matrix.loc[user].notna()].values  # Use predictions only for rated items

    if len(true_ratings) > 1:  # Ensure there's enough data for ranking
        ndcg_scores_voisins.append(ndcg(true_ratings, predicted_ratings))

# 5. Compute average NDCG
average_ndcg_user = np.mean(ndcg_scores_user)
average_ndcg_item = np.mean(ndcg_scores_item)
average_ndcg_bayes = np.mean(ndcg_scores_bayes)
average_ndcg_voisins = np.mean(ndcg_scores_voisins)

print(f"Average NDCG for User Mean approach: {average_ndcg_user:.4f}")
print(f"Average NDCG for Item Mean approach: {average_ndcg_item:.4f}")
print(f"Average NDCG for Bayes approach: {average_ndcg_bayes:.4f}")
print(f"Average NDCG for voisins approach: {average_ndcg_voisins:.4f}")


Average NDCG for User Mean approach: 0.6851
Average NDCG for Item Mean approach: 0.8653
Average NDCG for Bayes approach: 0.8870
Average NDCG for voisins approach: 0.9163


# Questions à réponse

### 1. Quelle serait selon vous la valeur NDCG d'un vote aléatoire? Du vote populaire (les films les plus populaires)?

Un vote aléatoire donnerait un NDCG très faible, proche de zéro, car il ne suit aucune logique ou préférence d'utilisateur. Les recommandations seraient complètement indépendantes des intérêts ou des préférences des utilisateurs, ce qui entraînerait une absence de pertinence dans les films recommandés. En revanche, un vote populaire, où les films les plus populaires sont recommandés en fonction des notes moyennes de tous les utilisateurs, pourrait générer un NDCG plus élevé, car ces films sont souvent appréciés par une majorité d’utilisateurs. Cependant, le NDCG ne sera pas nécessairement optimal, car les films populaires ne correspondent pas toujours aux préférences spécifiques de chaque utilisateur.



### 2. L'approche bayésienne devrait-elle donner un plus faible NDCG que l'approche u-u ? Devrait-on alors toujours prioriser u-u et laisser tomber l'approche bayésienne?

L'approche utilisateur-utilisateur (u-u) peut en effet donner un NDCG plus élevé que l’approche bayésienne dans des situations où les utilisateurs ont des comportements similaires et que les données sont suffisantes pour calculer ces similarités. Cela permet de recommander des films qui sont plus en phase avec les préférences des utilisateurs. Cependant, l'approche bayésienne reste pertinente, notamment dans des contextes où les données sont rares ou incertaines. Elle peut apporter une robustesse supplémentaire en estimant les préférences des utilisateurs même avec peu de données, ce qui permet de mieux gérer les situations d’incertitude. Il n'est donc pas toujours nécessaire de privilégier l'approche u-u au détriment de la bayésienne, car chaque approche a ses avantages selon le contexte.



### 3. La mesure du NDCG offre-t-elle une solution plus avantageuse que l'erreur quadratique ou absolue à l'évaluation de la valeur d'une liste de recommandations vis-à-vis de la satisfaction utilisateur?

Le NDCG est généralement plus avantageux que l'erreur quadratique ou absolue pour évaluer les systèmes de recommandation, car il prend en compte non seulement la précision des prédictions, mais aussi l’ordre des éléments dans la liste de recommandations, ce qui est crucial pour la satisfaction de l’utilisateur. En effet, les utilisateurs sont plus intéressés par les meilleurs films en haut de la liste que par des prédictions précises sur des films moins pertinents. Tandis que les erreurs quadratiques ou absolues mesurent la différence entre les valeurs prédites et réelles, sans se soucier de l’ordre ou de la pertinence des recommandations, le NDCG offre une mesure qui reflète mieux l’efficacité d’un système de recommandation en termes de satisfaction utilisateur, car il valorise les recommandations les plus pertinentes et bien positionnées.
